In [2]:
import cv2
import numpy as np
import datetime
import os

In [3]:
cam =cv2.VideoCapture(0)

In [4]:
cam.set(3,640) #set width as 640
cam.set(4,480) #set height as 480
scale=3 #resize of the image
wP=210*scale #width of the papper
hP=297*scale #height of the papper

In [5]:
#Save a frame for measurement
def save_frame(frame):
    now=datetime.datetime.now()
    filename = "frames/frame_{}.png".format(now.strftime("%Y-%m-%d_%H-%M-%S"))
    cv2.imwrite(filename, frame)

In [6]:
def blur(frame):
    frame=cv2.GaussianBlur(frame,(5,5),3)#image, kernel_size and std_dev(σ)
    return frame
# print("\u03c3")

In [7]:
def gray(frame):
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return(frame)

In [8]:
def edges(frame):
    frame=cv2.Canny(frame, 50, 50, apertureSize=3, L2gradient=False) #appearture is the sobel kernel 
    #L2gradient give accurate edge to frame
    contours, _= cv2.findContours(frame, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
 

    contour_img=cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_img, contours, -1, (0,0,255),3)


    # print("rect_top{},rect_bott{}, W{},H{}".format(topleft_axes, bottomright_axes, width, height))
    return contour_img


In [9]:
def eradicate(smaller_contour):
    smaller_contour=cv2.cvtColor(smaller_contour, cv2.COLOR_RGB2GRAY)
    contours, _= cv2.findContours(smaller_contour, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    big_contour= max(contours, key=cv2.contourArea)
    mask = cv2.drawContours(np.zeros(smaller_contour.shape[:2], dtype=np.uint8), [big_contour], -1, 255, -1)
    result=cv2.bitwise_and(smaller_contour, smaller_contour, mask=mask)
    return result

In [10]:
def min_arearect(frame):
    points=cv2.findNonZero(frame)
    rect=cv2.minAreaRect(points)#minimum area rotated rectangular box around contours
    box=cv2.boxPoints(rect)#obtaining 4 cornerpoints of rotated rectangle as [p1,p2,p3,p4]
    box=np.int0(box)
    # print("Boooooooox", box)
    x,y=box[0]
    w=box[2][0]-x
    h=box[2][1]-y
    paper_axes=[(x,y),(x+w, y+h)]
    cv2.drawContours(frame, [box], 0, (0,0,255),1) #3-> index of contour
    return frame, paper_axes


In [11]:
def drawrect(frame1, min_arearect):
    cords=min_arearect
    frame1=cv2.rectangle(frame1, cords[0], cords[1], (255,0,0), 2)
    return frame1

In [12]:
def remove_papper(frame):
    # convert2cv_u=cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    contours, _= cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    max_area = 0
    max_contour = None
    for cnt in contours:
          area = cv2.contourArea(cnt)
          if area>max_area:
            max_area=area
            max_contour= cnt
    framess=np.zeros_like(frame)
    mask = cv2.drawContours(framess, [max_contour], 0, 255, 1)
    frame=cv2.drawContours(frame, [max_contour], 0, 0, 10)
 
    return frame

In [13]:
def detect(object):
    contours, _= cv2.findContours(object[0], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # cv2.drawContours(object, contours, 0, 255, 5)
    
  
    cv2.drawContours(object, contours, -1, (0,0,255),3)

    object=min_arearect(object)

    # print("obj dimension", object[1])

    return object


In [14]:
def dimension_calc(papper, object):
    papper_pixel_width=papper[1][0]-papper[0][0]
    papper_pixel_height=papper[0][1]-papper[1][1]
    object_pixel_width=object[1][0]-object[0][0]
    print("Object pixel width is" , object_pixel_width)
    object_pixel_height=object[1][1]-object[0][1]
    print("Object pixel height is" , object_pixel_height)

    
    a_cm_pixel_width=papper_pixel_width/21
    print("a_cm_pixel_width", a_cm_pixel_width)
    a_cm_pixel_height=papper_pixel_height/29.7
    print("a_cm_pixel_height", a_cm_pixel_height)
    object_width_cm=object_pixel_width/a_cm_pixel_width
    object_height_cm=object_pixel_height/a_cm_pixel_height
    if object_height_cm<0:
        object_height_cm=object_height_cm*-1
    elif object_width_cm<0:
        object_width_cm=object_width_cm*-1

    return object_width_cm, object_height_cm



In [15]:
def display_dim(obj_wh, width, height, frame1):
    x,y=obj_wh[0]
    x1,y1=obj_wh[1]
    width_text=str(width)
    w_xpos=x+int(width*5.5714)//2
    w_ypos=y-10
    cv2.putText(frame1, width_text, (w_xpos, w_ypos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
 
    height_txt=str(height)
    h_xpos=x1+10
    h_ypos=y1-int(height*4.8822)//2
    cv2.putText(frame1, height_txt, (h_xpos, h_ypos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

    return frame1

In [16]:
while True:
    check, frame = cam.read()#capture frame from video
    if not check:
        break
    # frame=cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    # frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame=cv2.resize(frame,(320,240))
    frame1=frame
    frame=blur(frame)#showed err when converted to rgb error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
    frame=gray(frame)
    frame=edges(frame)
    frame=eradicate(frame)
    frame=min_arearect(frame)[0]
    papper_w_h=min_arearect(frame)[1]
    print("paper dim: ", papper_w_h)

    frame1=drawrect(frame1, min_arearect(frame)[1])
    frame=remove_papper(frame)
    frame=detect(frame)[0]
    obj_wh=detect(frame)[1]
    print("obj dim: ", obj_wh)
    frame1=drawrect(frame1, detect(frame)[1])

    dimension_calc(papper_w_h, obj_wh)
    width=dimension_calc(papper_w_h, obj_wh)[0]
    height=dimension_calc(papper_w_h, obj_wh)[1]
    frame1=display_dim(obj_wh, width, height, frame1)
    text="The Width {} and Height {} ".format(width, height)
    print(text)

    #Normalisation
    # frame= frame/255
    # Add an extra dimension at position 0
    #frame = np.expand_dims(frame, axis=0)
    cv2.imshow('video', frame)
    cv2.imshow("original", frame1)
    key=cv2.waitKey(1)
    if key == ord('s'):
        save_frame(frame)
    elif key==27 :
        break
cam.release()
cv2.destroyAllWindows()

/tmp/ipykernel_6043/4023814372.py:5: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box=np.int0(box)


paper dim:  [(162, 29), (277, 174)]
obj dim:  [(197, 55), (227, 120)]
Object pixel width is 30
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height -4.882154882154882
Object pixel width is 30
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height -4.882154882154882
Object pixel width is 30
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height -4.882154882154882
The Width 5.478260869565218 and Height 13.313793103448276 
paper dim:  [(161, 173), (276, 28)]
obj dim:  [(198, 55), (227, 120)]
Object pixel width is 29
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height 4.882154882154882
Object pixel width is 29
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height 4.882154882154882
Object pixel width is 29
Object pixel height is 65
a_cm_pixel_width 5.476190476190476
a_cm_pixel_height 4.882154882154882
The Width 5.2956521739130435 and Height 13.313793103448276 
paper 

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/convhull.cpp:143: error: (-215:Assertion failed) total >= 0 && (depth == CV_32F || depth == CV_32S) in function 'convexHull'


: 

In [62]:
#get images
path_to_image=("/home/agj/VSCODE/Jeffrin/Project obj-measur-sys/frames/")
def get_latest_image(directory, valid_extensions=('jpg', 'jpeg', 'png')):
    #get filepaths of all files and dirs in the given dir
    valid_files=[os.path.join(directory, filename) for filename in os.listdir(directory)]
    valid_files=[f for f in valid_files if '.' in f and f.rsplit('.',1)[1] in valid_extensions and os.path.isfile(f)]
    #f.rsplit('.',1)[1] split the string once and select the string after ".".
    #os.path.isfile(f) checks the string f corresponds to a valid file or not.
    if not valid_files:
        raise ValueError("No valid images in %s" % directory)
    return max(valid_files, key=os.path.getmtime)
last_image=get_latest_image(path_to_image)

In [68]:
#read image
imge=cv2.imread(last_image)
imge_cpy=imge.copy()
imge1=imge.astype(np.uint8)
print(type(imge1))
cv2.imshow('image', imge)
cv2.imshow('Img_cpy', imge_cpy)
cv2.waitKey(5000)
cv2.destroyAllWindows()

<class 'numpy.ndarray'>


In [71]:
gray=cv2.cvtColor(imge, cv2.COLOR_BGR2GRAY)

In [85]:
contours, _= cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# _, thresh=cv2.threshold(imge, 127, 255, cv2.THRESH_BINARY)
# # Loop through contours
# for cnt in contours:
#     if cv2.contourArea(cnt)< .5 :
#         cv2.drawContours(thresh, [cnt], 0, 0, -1)
    

# cv2.imshow('Out_put Image', thresh)
cv2.imshow("rec", gray)

cv2.waitKey(50000)
cv2.destroyAllWindows()

In [86]:
#erasing all pixes outside big contour
big_contour= max(contours, key=cv2.contourArea)
mask = cv2.drawContours(np.zeros(gray.shape[:2], dtype=np.uint8), [big_contour], -1, 255, -1)
result=cv2.bitwise_and(gray, gray, mask=mask)
cv2.imshow('result', result)
cv2.waitKey(50000)
cv2.destroyAllWindows()